## Setup do GLFW, shaders e vértices:

In [10]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
from time import sleep # background color fade
import math # calculo com sin e cos

glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 600, "Exercicio 2 by Joao Villaca", None, None)
glfw.make_context_current(window)

vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")
    
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

# calculo dos vertices do circulo
num_vertices = 64 # define a "qualidade" do circulo
pi = 3.14
counter = 0
radius = 0.40
vertices = np.zeros(num_vertices + 11, [("position", np.float32, 2)])

angle = 0.0
for counter in range(num_vertices):
    angle += 2*pi/num_vertices 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[counter] = [x,y]

#print(vertices) # debug

# colocando os vertices do quadrado
vertices[num_vertices] = [+0.4, -0.4]
vertices[num_vertices + 1] = [+0.4, +0.4]
vertices[num_vertices + 2] = [-0.4, -0.4]
vertices[num_vertices + 3] = [-0.4, +0.4]
#print(vertices) # debug

# vertices do triangulo
vertices[num_vertices + 4] = [+0.4, -0.4]
vertices[num_vertices + 5] = [-0.4, -0.4]
vertices[num_vertices + 6] = [-0.0, +0.4]
#print(vertices) # debug

# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

loc_color = glGetUniformLocation(program, "color")    

# Após o setup, execute uma das seguintes janelas:

## Triângulo + mudança de escala com mouse esquerdo/direito:

In [2]:
# variaveis de mudança de escala
sx = 1.0
sy = 1.0

def mouse_listener(window, button, action, mods):
    
#    print('[mouse event] button=',button)
#    print('[mouse event] action=',action)
#    print('[mouse event] mods=',mods)
#    print('-------')   

    global sx, sy
    
    # comandos de mudança de escala
    if button == 0:
        sx *= 1.03
        sy *= 1.03
    if button == 1:
        sx *= 0.97
        sy *= 0.97

glfw.set_mouse_button_callback(window, mouse_listener)
glfw.show_window(window)

R = 0.0
G = 0.0
B = 0.0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0, 0, 0, 1.0)
    
    # matriz de mudança de escala
    mat_escala =    np.array([      sx, 0.0, 0.0, 0.0, 
                                    0.0, sy, 0.0, 0.0, 
                                    0.0, 0.0, 0.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    
    glUniform4f(loc_color, 0, 1, 0, 1.0); # modificando a cor do triângulo
    glDrawArrays(GL_TRIANGLE_STRIP, num_vertices + 4, 3)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_escala)
    
    glfw.swap_buffers(window)

glfw.terminate()

## Quadrado + rotação com as setas esquerda e direita:

In [4]:
rotate = 0.0

def key_listener(window, key, scancode, action, mods):
    
#    print('[key event] key=',key)
#    print('[key event] scancode=',scancode)
#    print('[key event] action=',action)
#    print('[key event] mods=',mods)
#    print('-------')

    global rotate
    
    # comandos de rotação
    if key == 262 and action > 0: rotate += 1.0
    if key == 263 and action > 0: rotate -= 1.0

glfw.set_key_callback(window, key_listener)
glfw.show_window(window)

R = 0.0
G = 0.0
B = 0.0
while not glfw.window_should_close(window):

    sleep(0.05)
    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0, 0, 0, 1.0)
    
    # matriz de rotação
    mat_rotation = np.array([       math.cos(rotate), -(math.sin(rotate)), 0.0, 0.0, 
                                    math.sin(rotate), math.cos(rotate), 0.0, 0.0, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    glUniform4f(loc_color, 0, 0, 1, 1.0); # modificando a cor do quadrado
    glDrawArrays(GL_TRIANGLE_STRIP, num_vertices, 4)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_rotation)
    
    glfw.swap_buffers(window)
    
glfw.terminate()

## Círculo + translação com as teclas WASD:

In [7]:
# variaveis de translação
t_x = 0
t_y = 0

def key_listener(window, key, scancode, action, mods):
    
#     print('[key event] key=',key)
#     print('[key event] scancode=',scancode)
#     print('[key event] action=',action)
#     print('[key event] mods=',mods)
#     print('-------')

    global t_x, t_y
    
    # comandos de translação
    if key == 87: t_y += 0.03 #cima
    if key == 83: t_y -= 0.03 #baixo
    if key == 65: t_x -= 0.03 #esquerda
    if key == 68: t_x += 0.03 #direita

glfw.set_key_callback(window, key_listener)
glfw.show_window(window)

R = 0.0
G = 0.0
B = 0.0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0, 0, 0, 1.0)
    
    # matriz de translação
    mat_translation = np.array([    1.0, 0.0, 0.0, t_x, 
                                    0.0, 1.0, 0.0, t_y, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    glUniform4f(loc_color, 1, 0, 0, 1.0); # modificando a cor do circulo
    glDrawArrays(GL_TRIANGLE_FAN, 0, num_vertices)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_translation)
    
    glfw.swap_buffers(window)
    
glfw.terminate()

## Triângulo com mudança de escala e quadrado com translação:

In [6]:
# variaveis de translação
t_x = 0
t_y = 0

def key_listener(window, key, scancode, action, mods):
    
#     print('[key event] key=',key)
#     print('[key event] scancode=',scancode)
#     print('[key event] action=',action)
#     print('[key event] mods=',mods)
#     print('-------')

    global t_x, t_y
    
    # comandos de translação
    if key == 87: t_y += 0.03 #cima
    if key == 83: t_y -= 0.03 #baixo
    if key == 65: t_x -= 0.03 #esquerda
    if key == 68: t_x += 0.03 #direita
        
# variaveis de mudança de escala
sx = 1.0
sy = 1.0

def mouse_listener(window, button, action, mods):
    
#    print('[mouse event] button=',button)
#    print('[mouse event] action=',action)
#    print('[mouse event] mods=',mods)
#    print('-------')   

    global sx, sy
    
    # comandos de mudança de escala
    if button == 0:
        sx *= 1.03
        sy *= 1.03
    if button == 1:
        sx *= 0.97
        sy *= 0.97

glfw.set_key_callback(window, key_listener)
glfw.set_mouse_button_callback(window, mouse_listener)
glfw.show_window(window)

R = 0.0
G = 0.0
B = 0.0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0, 0, 0, 1.0)
    
    # matriz de translação
    mat_translation = np.array([    1.0, 0.0, 0.0, t_x, 
                                    0.0, 1.0, 0.0, t_y, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    
    # matriz de mudança de escala
    mat_escala =    np.array([      sx, 0.0, 0.0, 0.0, 
                                    0.0, sy, 0.0, 0.0, 
                                    0.0, 0.0, 0.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_translation)
    glUniform4f(loc_color, 0, 0, 1, 1.0); # modificando a cor do quadrado
    glDrawArrays(GL_TRIANGLE_STRIP, num_vertices, 4)
    
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_escala)
    glUniform4f(loc_color, 0, 1, 0, 1.0); # modificando a cor do triângulo
    glDrawArrays(GL_TRIANGLE_STRIP, num_vertices + 4, 3)
    
    glfw.swap_buffers(window)
    
glfw.terminate()

## Objeto com todas as transformações geométricas

In [11]:
# variaveis de translação
t_x = 0
t_y = 0
rotate = 0.0

def key_listener(window, key, scancode, action, mods):
    
#     print('[key event] key=',key)
#     print('[key event] scancode=',scancode)
#     print('[key event] action=',action)
#     print('[key event] mods=',mods)
#     print('-------')

    global t_x, t_y, rotate
    
    # comandos de translação
    if key == 87: t_y += 0.03 #cima
    if key == 83: t_y -= 0.03 #baixo
    if key == 65: t_x -= 0.03 #esquerda
    if key == 68: t_x += 0.03 #direita
        
    # comandos de rotação
    if key == 262 and action > 0: rotate += 1.0
    if key == 263 and action > 0: rotate -= 1.0
        
# variaveis de mudança de escala
sx = 1.0
sy = 1.0

def mouse_listener(window, button, action, mods):
    
    #print('[mouse event] button=',button)
    #print('[mouse event] action=',action)
    #print('[mouse event] mods=',mods)
    #print('-------')   

    global sx, sy
    
    # comandos de mudança de escala
    if button == 0 and action == 1:
        sx *= 1.03
        sy *= 1.03
    if button == 1 and action == 1:
        sx *= 0.97
        sy *= 0.97

glfw.set_key_callback(window, key_listener)
glfw.set_mouse_button_callback(window, mouse_listener)
glfw.show_window(window)

R = 0.0
G = 0.0
B = 0.0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0, 0, 0, 1.0)
    
    # matriz de translação
    mat_translation = np.array([[1.0, 0.0, 0.0, t_x], [0.0, 1.0, 0.0, t_y],
                                [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    
    # matriz de mudança de escala
    mat_escala = np.array([[sx, 0.0, 0.0, 0.0], [0.0, sy, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    
    # matriz de rotação
    mat_rotation = np.array([[math.cos(rotate), -(math.sin(rotate)), 0.0, 0.0],
                             [math.sin(rotate), math.cos(rotate), 0.0, 0.0],
                             [0, 0, 1, 0],
                             [0, 0, 0, 1]])
    
    #print(mat_translation)
    #print(mat_escala)
    #print(mat_rotation)
 
    # matriz T x E x R    
    mat_all = mat_translation@mat_escala@mat_rotation

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_all)
    glUniform4f(loc_color, 0, 0, 1, 1.0); # modificando a cor do quadrado
    glDrawArrays(GL_TRIANGLE_STRIP, num_vertices, 4)

    
    glfw.swap_buffers(window)
    
glfw.terminate()